In [ ]:
!nvidia-smi

Fri Sep 20 16:38:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Aula 12

In [6]:
%%writefile exercicio_p1.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/extrema.h>
#include <iostream>
#include <chrono>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <thrust/sequence.h>
using namespace std;



struct saxpy
{
  int a;
  saxpy(int a_) : a(a_) {};

  __host__ __device__
  double operator()(const double& x, const double& y) const{
    return x*a + y;
  }
};


int main(){


  thrust::device_vector<double> v1(10,0);
  thrust::sequence(v1.begin(),v1.end());

  thrust::device_vector<double> v2(10,5);

  thrust::device_vector<double> v3(10,0);

  thrust::transform(v1.begin(),v1.end(),v2.begin(), v3.begin(),saxpy(3));

  printf("v3: ");
  for (thrust::device_vector<double>::iterator i = v3.begin(); i != v3.end(); i++) {
      std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
  };



  return 0;
}


Overwriting exercicio_p1.cu


In [7]:
!nvcc -arch=sm_75 -std=c++14 exercicio_p1.cu -o exercicio_p1

In [8]:
!./exercicio_p1

v3: 5 8 11 14 17 20 23 26 29 32 

In [57]:
%%writefile exercicio_p2.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <iostream>
#include <chrono>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <thrust/sequence.h>
using namespace std;

struct square{
  __host__ __device__
  float operator()(const float& x) const{
    return x*x;
  }
};

float magnitude(thrust::device_vector<float>& v){
    float soma_quadrados = thrust::transform_reduce(v.begin(),v.end(),square(),0.0f, thrust::plus<float>());
    return std::sqrt(soma_quadrados);
  };

int main(){

  //usando GPU


  thrust::device_vector<float> v1(1000,0);
  thrust::sequence(v1.begin(),v1.end());

  std::chrono::duration<double> time1;
  auto t1 = std::chrono::steady_clock::now();
  float mag = magnitude(v1);
  auto t2 =std::chrono::steady_clock::now();
  time1 = t2-t1;

  cout << "Magnitude GPU: " << mag << endl;
  cout << "Tempo GPU: " << time1.count() << " s" <<endl;



  return 0;
}

Overwriting exercicio_p2.cu


In [58]:
!nvcc -arch=sm_75 -std=c++14 exercicio_p2.cu -o exercicio_p2

In [59]:
!./exercicio_p2


Magnitude GPU: 18243.7
Tempo GPU: 6.8155e-05 s


Magnitude (CPU): 18243.7

Tempo (CPU): 8.8e-06 s

código para rodar está no repositório, aula 12.

A diferença entre os tempos pode se explicada pelo fato de, primeiro, estarmos lidando com vetores pequenos, de apenas 1000 elementos, o que já equipara os cálculos na cpu e na gpu, já que a gpu não é muito eficiente para pequenas entradas. Segundo, a invocação de kernels, gerenciamento de threads dentre outras coisas que acontecem dentro da gpu, pode ser responsável pela diferença de tempo analisada.

In [69]:
%%writefile exercicio_p3.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <iostream>
#include <chrono>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <thrust/sequence.h>
using namespace std;


struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}  // Construtor que armazena a média

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;  // Diferença em relação à média
        return diff * diff;  // Elevar ao quadrado a diferença
    }
};

float calculate_variance(const thrust::device_vector<float>& vec, float mean) {
    return thrust::transform_reduce(vec.begin(), vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / vec.size();
}

float calculate_mean_var(const thrust::device_vector<float>& vec){
  float mean = thrust::reduce(vec.begin(),vec.end(), 0.0f, thrust::plus<float>()) / vec.size();
  return thrust::transform_reduce(vec.begin(), vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / vec.size();
}


int main(){

  thrust::device_vector<float> vec(1000,0);
  thrust::sequence(vec.begin(),vec.end());


  float mean = thrust::reduce(vec.begin(),vec.end(), 0.0f, thrust::plus<float>()) / vec.size();
  //fusion kernel
  std::chrono::duration<double> time1;
  auto t1 = std::chrono::steady_clock::now();

  float variancia_fusion = calculate_variance(vec,mean);

  auto t2 =std::chrono::steady_clock::now();
  time1 = t2-t1;

  cout << "variancia com fusion: " << variancia_fusion << endl;
  cout << "Tempo: " << time1.count() << " s" <<endl;


  //sem fusion
  std::chrono::duration<double> time2;
  auto t3 = std::chrono::steady_clock::now();

  float variancia = calculate_mean_var(vec);

  auto t4 =std::chrono::steady_clock::now();
  time2 = t4-t3;

  cout << "variancia com fusion: " << variancia << endl;
  cout << "Tempo: " << time2.count() << " s" <<endl;

  return 0;
}



Overwriting exercicio_p3.cu


In [70]:
!nvcc -arch=sm_75 -std=c++14 exercicio_p3.cu -o exercicio_p3

In [71]:
!./exercicio_p3

variancia com fusion: 83333.2
Tempo: 7.5557e-05 s
variancia com fusion: 83333.2
Tempo: 0.000105467 s


confesso que fiquei confuso. Não tenho certeza se entedi corretamente a diferença entre as funções usando fusion e sem usá-lo. No final, o cálculo da média terá que ser realizado da mesma forma...